In [1]:
%cd '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect'

/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect


In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import cv2
import os
import sys
import numpy as np
import random

sys.path.append(os.getcwd())

from cnn import make_model

In [50]:
def load_all_models(save_path, features):
	all_models = list()
	for feature in features:
		filename = save_path + str(feature) + '/save.h5'
		model = tf.keras.models.load_model(filename)
		all_models.append(model)
		print('loaded model of ' + str(feature))
	return all_models

def define_stacked_model(neural_nets, features):

	for i in range(len(neural_nets)):
		model = neural_nets[i]
		for layer in model.layers:
			layer.trainable = False
			layer._name = 'ensemble_' + str(features[i]) + '_' + layer.name
	# define multi-headed input
	ensemble_visible = [model.input for model in neural_nets]
	i = 0
	for layer in ensemble_visible:
		layer.name = str(features[i]) + "_" + layer.name 
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in neural_nets]
	i = 0
	for layer in ensemble_outputs:
		layer.name = str(features[i]) + "_" + layer.name 
    
	merge = tf.keras.layers.concatenate(ensemble_outputs)
	hidden = tf.keras.layers.Dense(10, activation='relu')(merge)
	output = tf.keras.layers.Dense(3, activation='softmax')(hidden)
	model = tf.keras.Model(inputs=ensemble_visible, outputs=output)
	# plot graph of ensemble
	tf.keras.utils.plot_model(model, show_shapes=True, to_file='data/plots/model_graph.png')
	# compile
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# encode output data
	inputy_enc = to_categorical(inputy)
	# fit model
	model.fit(X, inputy_enc, epochs=300, verbose=0)
 
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# make prediction
	return model.predict(X, verbose=0)

In [61]:
if __name__ == "__main__":

    save_path = 'categorization/model_saves/'
    face_features = ["mouth", "face", "skin", "eyes"]

    all_models = load_all_models(save_path, face_features)
    # stacked = define_stacked_model(all_models, face_features)


loaded model of mouth
loaded model of face
loaded model of skin
loaded model of eyes


In [72]:
ensemble_visible = [model.input for model in all_models]

i = 0
for model in all_models:
    model.input._name = model.input.name + "_" + face_features[i]
    ensemble_visible.append(model.input)
    i += 1

In [73]:

ensemble_outputs = [model.output for model in all_models]
    
merge = tf.keras.layers.concatenate(ensemble_outputs)
hidden = tf.keras.layers.Dense(10, activation='relu')(merge)
output = tf.keras.layers.Dense(3, activation='softmax')(hidden)

model = tf.keras.Model(inputs=ensemble_visible, outputs=output)

ValueError: The name "conv2d_input" is used 3 times in the model. All layer names should be unique.